In [ ]:
! pip install ipdb

     |████████████████████████████████| 791 kB 34.1 MB/s 
     |████████████████████████████████| 374 kB 59.4 MB/s 
  Created wheel for ipdb: filename=ipdb-0.13.9-py3-none-any.whl size=11648 sha256=861022cb8ba5abc90c726ae1d8973e672954ef3d7d00e2ea212c614cedf9855f
  Stored in directory: /root/.cache/pip/wheels/65/cd/cc/aaf92acae337a28fdd2aa4d632196a59745c8c39f76eaeed01
Successfully built ipdb
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting uninstall: ipython
    Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit

In [78]:
! pip install torchmetrics

     |████████████████████████████████| 332 kB 30.2 MB/s 


In [88]:
import pandas as pd
import numpy as np
from string import punctuation
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim
import ipdb

import re
import torchmetrics
from torchmetrics import F1
from torchmetrics.functional import f1, recall
import matplotlib.pyplot as plt

In [38]:
pos_tweets = pd.read_csv('positive (1).csv', encoding='utf-8', sep=';', header=None,  names=[0,1,2,'text','tone',5,6,7,8,9,10,11])

In [39]:
neg_tweets = pd.read_csv('negative (1).csv', encoding='utf-8', sep=';', header=None, names=[0,1,2,'text','tone',5,6,7,8,9,10,11] )
neg_tweets['tone'] = 0

In [40]:
all_tweets_data = pos_tweets.append(neg_tweets)
print(len(all_tweets_data))

226834


In [41]:
tweets_data = shuffle(all_tweets_data[['text','tone']])[:100000]

In [42]:
def preprocess(text):
    res = re.sub(r'[^А-яЁё]', ' ', text)
    res = res.lower()
    return res

In [43]:
tweets_data['text'] = tweets_data.text.apply(preprocess)

In [44]:
train_sentences, val_sentences = train_test_split(tweets_data, test_size=0.2)

In [69]:
vocab = Counter()

for text in tweets_data['text']:
    text = text.split()
    vocab.update(text)
print('всего уникальных токенов:', len(vocab))

filtered_vocab = set()
for word in vocab:
    if vocab[word] > 5:
        filtered_vocab.add(word)
print('уникальных токенов, встретившихся больше 5 раз:', len(filtered_vocab))

всего уникальных токенов: 106408
уникальных токенов, встретившихся больше 5 раз: 14485


In [46]:
#создаем словарь с индексами symbol2id, для спецсимвола паддинга дефолтный индекс - 0
symbol2id = {'PAD':0}

for symbol in filtered_vocab:
    symbol2id[symbol] = len(symbol2id)

#обратный словарь для того, чтобы раскодировать последовательность
id2symbol = {i:symbol for symbol, i in symbol2id.items()}

In [47]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cpu')

Dataset & DataLoader

In [48]:
class TweetsDataset(Dataset):

    def __init__(self, dataset, word2id, DEVICE):
        self.dataset = dataset['text'].values
        self.word2id = word2id
        self.length = dataset.shape[0]
        self.target = torch.Tensor(dataset['tone'].values)
        self.device = DEVICE

    def __len__(self): #это обязательный метод, он должен уметь считать длину датасета
        return self.length

    def __getitem__(self, index): #еще один обязательный метод. По индексу возвращает элемент выборки
        tokens = self.dataset[index].split()
        ids = torch.LongTensor([self.word2id[token] for token in tokens if token in self.word2id])
        y = [self.target[index]]
        return ids, y
    
#    def preprocess(self, text):
#        tokens = text.lower().split()
#        tokens = [token.strip(punctuation) for token in tokens]
#        tokens = [token for token in tokens if token]
#        return tokens

    def collate_fn(self, batch): #этот метод можно реализовывать и отдельно,
    # он понадобится для DataLoader во время итерации по батчам
      ids, y = list(zip(*batch))
      padded_ids = pad_sequence(ids, batch_first=True).to(self.device)
      #мы хотим применять BCELoss, он будет брать на вход predicted размера batch_size x 1 (так как для каждого семпла модель будет отдавать одно число), target размера batch_size x 1
      y = torch.Tensor(y).to(self.device) # tuple ([1], [0], [1])  -> Tensor [[1.], [0.], [1.]] 
      return padded_ids, y

In [49]:
train_dataset = TweetsDataset(train_sentences, symbol2id, DEVICE)
train_sampler = RandomSampler(train_dataset)
train_iterator = DataLoader(train_dataset, collate_fn = train_dataset.collate_fn, sampler=train_sampler, batch_size=1024)

In [50]:
batch = next(iter(train_iterator))

In [52]:
val_dataset = TweetsDataset(val_sentences, symbol2id, DEVICE)
val_sampler = SequentialSampler(val_dataset)
val_iterator = DataLoader(val_dataset, collate_fn = val_dataset.collate_fn, sampler=val_sampler, batch_size=1024)

In [53]:
test_batch = next(iter(val_iterator))

CNN

In [ ]:
self.bigrams = nn.Conv1d(in_channels=embedding_dim, out_channels=60, kernel_size=2, padding='same')
self.trigrams = nn.Conv1d(in_channels=embedding_dim, out_channels=100, kernel_size=3, padding='same')
self.sec_conv = nn.Conv1d(in_channels=160, out_channels=40, kernel_size=3, padding='same')

In [125]:
class CNN(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.bigrams = nn.Conv1d(in_channels=embedding_dim, out_channels=60, kernel_size=2, padding='same')
        self.trigrams = nn.Conv1d(in_channels=embedding_dim, out_channels=100, kernel_size=3, padding='same')
        self.sec_conv = nn.Conv1d(in_channels=160, out_channels=40, kernel_size=3, padding='same')
        self.pooling = nn.MaxPool1d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.hidden = nn.Linear(in_features=40, out_features=1)
        self.dropout = nn.Dropout(p=0.5)
        self.out = nn.Sigmoid()

    def forward(self, word):
        #batch_size x seq_len
        embedded = self.embedding(word)
        #batch_size x seq_len x embedding_dim
        embedded = embedded.transpose(1,2)
        #batch_size x embedding_dim x seq_len
        feature_map_bigrams = self.dropout(self.relu(self.bigrams(embedded)))
        #batch_size x filter_count2 x seq_len* 
        feature_map_trigrams = self.dropout(self.relu(self.trigrams(embedded)))
        #batch_size x filter_count3 x seq_len*

        #pooling1 = feature_map_bigrams.max(2)[0] 
        # batch_size x filter_count2
        #pooling2 = feature_map_trigrams.max(2)[0]
        # batch_size x filter_count3
        concat = torch.cat((feature_map_bigrams, feature_map_trigrams), 1)
        third_conv = self.dropout(self.pooling(self.relu(self.sec_conv(concat))))
        pooling1 = third_conv.max(2)[0]
        # batch _size x (filter_count2 + filter_count3)
        logits = self.hidden(pooling1) 
        logits = self.out(self.dropout(logits))      
        return logits

Train

In [71]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0 # для подсчета среднего лосса на всех батчах

    model.train()  # ставим модель в обучение, явно указываем, что сейчас надо будет хранить градиенты у всех весов

    for i, (texts, ys) in enumerate(iterator): #итерируемся по батчам
        optimizer.zero_grad()  #обнуляем градиенты
        preds = model(texts)  #прогоняем данные через модель
        loss = criterion(preds, ys) #считаем значение функции потерь  
        loss.backward() #считаем градиенты  
        optimizer.step() #обновляем веса 
        epoch_loss += loss.item() #сохраняем значение функции потерь
        if not (i + 1) % int(len(iterator)/5):
            print(f'Train loss: {epoch_loss/i}')      
    return  epoch_loss / len(iterator) # возвращаем среднее значение лосса по всей выборке

In [72]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_metric = 0
    model.eval() 
    with torch.no_grad():
        for i, (texts, ys) in enumerate(iterator):   
            preds = model(texts)  # делаем предсказания на тесте
            loss = criterion(preds, ys)   # считаем значения функции ошибки для статистики  
            epoch_loss += loss.item()
            batch_metric = f1(preds.round().long(), ys.long(), ignore_index=0)
            epoch_metric += batch_metric

            if not (i + 1) % int(len(iterator)/5):
              print(f'Val loss: {epoch_loss/i}, Val f1: {epoch_metric/i}')
        
    return epoch_metric / len(iterator), epoch_loss / len(iterator) # возвращаем среднее значение по всей выборке

In [73]:
def learning(n_epochs, model, optimizer, criterion):
    for i in range(n_epochs):
        print(f'\nstarting Epoch {i}')
        print('Training...')
        epoch_loss = train(model, train_iterator, optimizer, criterion)
        losses.append(epoch_loss)
        print('\nEvaluating on train...')
        f1_on_train,_ = evaluate(model, train_iterator, criterion)
        f1s.append(f1_on_train.cpu())
        print('\nEvaluating on test...')
        f1_on_test, epoch_loss_on_test = evaluate(model, val_iterator, criterion)
        losses_eval.append(epoch_loss_on_test)
        f1s_eval.append(f1_on_test.cpu())

In [150]:
model = CNN(len(symbol2id), 4)
optimizer = optim.Adam(model.parameters(), lr=0.0005)
criterion = nn.BCELoss()  

# веса модели и значения лосса храним там же, где и все остальные тензоры
model = model.to(DEVICE)
criterion = criterion.to(DEVICE)

In [151]:
losses = []
losses_eval = []
f1s = []
f1s_eval = []

for i in range(30):
    print(f'\nstarting Epoch {i}')
    print('Training...')
    epoch_loss = train(model, train_iterator, optimizer, criterion)
    losses.append(epoch_loss)
    print('\nEvaluating on train...')
    f1_on_train,_ = evaluate(model, train_iterator, criterion)
    f1s.append(f1_on_train)
    print('\nEvaluating on test...')
    f1_on_test, epoch_loss_on_test = evaluate(model, val_iterator, criterion)
    losses_eval.append(epoch_loss_on_test)
    f1s_eval.append(f1_on_test)


starting Epoch 0
Training...
Train loss: 0.7660761943885258
Train loss: 0.7357284919968967
Train loss: 0.7252624210986224
Train loss: 0.7193151288113352
Train loss: 0.7155177802652926

Evaluating on train...
Val loss: 0.7457910222666604, Val f1: 0.017919911071658134
Val loss: 0.7194051680893734, Val f1: 0.01705903373658657
Val loss: 0.7110642343759537, Val f1: 0.0165686197578907
Val loss: 0.7073793189000275, Val f1: 0.01682894676923752
Val loss: 0.7048568306742488, Val f1: 0.0160882156342268

Evaluating on test...
Val loss: 0.9262000521024069, Val f1: 0.020141126587986946
Val loss: 0.793457567691803, Val f1: 0.015261885710060596
Val loss: 0.7569042064926841, Val f1: 0.016808820888400078
Val loss: 0.7403485298156738, Val f1: 0.01718178577721119
Val loss: 0.7309328411754809, Val f1: 0.01534013357013464

starting Epoch 1
Training...
Train loss: 0.7502446728093284
Train loss: 0.7232050135217863
Train loss: 0.7143856950781562
Train loss: 0.7102252739970967
Train loss: 0.707700131712733

Ev

In [83]:
from tqdm import tqdm


def predict(model, iterator):
    model.eval()
    fp = []
    fn = []
    tp = [] 
    tn = []
    with torch.no_grad():
        for i, (texts, ys) in tqdm(enumerate(iterator)):
            preds = model(texts)  # делаем предсказания на тесте
            for pred, gold, text in zip(preds, ys, texts):
                text = ' '.join([id2symbol[int(word)] for word in text if word !=0])
                if round(pred.item()) > gold:
                    fp.append(text)
                elif round(pred.item()) < gold:
                    fn.append(text)
                elif round(pred.item()) == gold == 1:
                    tp.append(text)
                elif round(pred.item()) == gold == 0:
                    tn.append(text)
    return fp, fn, tp, tn

In [154]:
fp, fn, tp, tn = predict(model, val_iterator)

20it [00:05,  3.85it/s]


In [155]:
accuracy = (len(tp)+len(tn))/(len(fp) + len(fn) + len(tp) + len(tn))

In [156]:
print(accuracy) #размер эмбеддинга 4

0.6404


In [145]:
print(accuracy) #размер эмбеддинга 6

0.64725


In [136]:
print(accuracy) #до изменений

0.6909


In [157]:
recall = len(tp)/(len(tp) + len(fn))

In [158]:
print(recall) #размер эмбеддинга 4

0.6628184713375797


In [147]:
print(recall) #размер эмбеддинга 6

0.7364649681528662


In [138]:
print(recall) #до изменений

0.6797372611464968


In [159]:
precision = len(tp)/(len(tp)+len(fp))

In [160]:
print(precision) #размер эмбеддинга 4

0.6364678899082569


In [149]:
print(precision) #размер эмбеддинга 6

0.6267468450918946


In [140]:
print(precision) #до изменений

0.6973657341229325


Вторая модель

In [167]:
vocab_sym = Counter()
for symbol in tweets_data['text']:
    vocab_sym.update(list(symbol))

filtered_vocab_sym = set()

for symb in vocab_sym:
    if vocab_sym[symb] > 5:
        filtered_vocab_sym.add(symb)

symb2id = {'PAD':0}

for symb in filtered_vocab_sym:
    symb2id[symb] = len(symb2id)
    
id2symbol = {i:symb for symb, i in symb2id.items()}

In [163]:
class TweetsDataset2(Dataset):

    def __init__(self, dataset, word2id, symb2id, DEVICE):
        self.dataset = dataset['text'].values
        self.word2id = word2id
        self.symb2id = symb2id
        self.length = dataset.shape[0]
        self.target = torch.Tensor(dataset['tone'].values)
        self.device = DEVICE

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        symbols = list(self.dataset[index])
        symbols_ids = torch.LongTensor([self.symb2id[symb] for symb in symbols if symb in self.symb2id])
        
        tokens = self.dataset[index].split()
        words_ids = torch.LongTensor([self.word2id[token] for token in tokens if token in self.word2id])
        y = [self.target[index]]
        
        return words_ids, symbols_ids, y

    def collate_fn(self, batch):
        words_ids, symbols_ids, y = list(zip(*batch))
        padded_sym_ids = pad_sequence(symbols_ids, batch_first=True).to(self.device)
        padded_words_ids = pad_sequence(words_ids, batch_first=True).to(self.device)
        y = torch.Tensor(y).to(self.device)
        return padded_words_ids, padded_sym_ids, y

In [165]:
train_dataset = TweetsDataset2(train_sentences, symbol2id, symb2id, DEVICE)
train_sampler = RandomSampler(train_dataset)
train_iterator = DataLoader(train_dataset, collate_fn = train_dataset.collate_fn, sampler=train_sampler, batch_size=1024)
val_dataset = TweetsDataset2(val_sentences, symbol2id, symb2id, DEVICE)
val_sampler = SequentialSampler(val_dataset)
val_iterator = DataLoader(val_dataset, collate_fn = val_dataset.collate_fn, sampler=val_sampler, batch_size=1024)